# PyTorch入門 - 自作Transformerを理解するための基礎

このノートブックでは、自作Transformerのソースコードを読み解くために必要なPyTorchの基礎を学びます。

## 目次

1. [テンソル（Tensor）の基礎](#1-テンソルの基礎)
2. [自動微分（Autograd）](#2-自動微分)
3. [nn.Moduleでモデルを作る](#3-nnmoduleでモデルを作る)
4. [よく使う層（Linear, Embedding, LayerNorm）](#4-よく使う層)
5. [活性化関数の選び方](#5-活性化関数の選び方)
6. [損失関数](#6-損失関数)
7. [最適化アルゴリズム](#7-最適化アルゴリズム)
8. [学習ループの書き方](#8-学習ループの書き方)
9. [推論（評価）モード](#9-推論モード)
10. [モデルの確認とデバッグ](#10-モデルの確認とデバッグ)
11. [GPUの使い方](#11-gpuの使い方)
12. [Transformerコードとの対応](#12-transformerコードとの対応)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# 日本語フォント設定
import matplotlib
matplotlib.rcParams['font.family'] = 'Hiragino Sans'
matplotlib.rcParams['axes.unicode_minus'] = False

# 再現性のためのシード固定
torch.manual_seed(42)

print(f"PyTorch version: {torch.__version__}")

---

## 1. テンソルの基礎

**テンソル（Tensor）** はPyTorchの基本データ構造です。NumPyの配列に似ていますが、GPUで計算でき、自動微分をサポートします。

### 1.1 テンソルの作成

In [ ]:
# 直接作成
a = torch.tensor([1, 2, 3])
print(f"1次元テンソル: {a}")
print(f"形状: {a.shape}")
print(f"データ型: {a.dtype}")
print()

# 2次元テンソル（行列）
b = torch.tensor([[1, 2], [3, 4], [5, 6]])
print(f"2次元テンソル:\n{b}")
print(f"形状: {b.shape}  # (行数, 列数)")
print()

# 3次元テンソル（Transformerで頻出）
c = torch.randn(2, 3, 4)  # (バッチ, シーケンス長, 特徴次元)
print(f"3次元テンソルの形状: {c.shape}")
print(f"  - バッチサイズ: {c.shape[0]}")
print(f"  - シーケンス長: {c.shape[1]}")
print(f"  - 特徴次元: {c.shape[2]}")

### 1.2 特殊なテンソルの作成

In [ ]:
# ゼロで初期化
zeros = torch.zeros(3, 4)
print(f"ゼロテンソル:\n{zeros}")

# 1で初期化
ones = torch.ones(2, 3)
print(f"\n1テンソル:\n{ones}")

# 乱数（標準正規分布）
randn = torch.randn(2, 3)  # 平均0、標準偏差1
print(f"\n正規分布乱数:\n{randn}")

# 整数乱数（トークンIDなどに使用）
randint = torch.randint(0, 10, (3, 4))  # 0-9の整数
print(f"\n整数乱数:\n{randint}")

# 単位行列
eye = torch.eye(3)
print(f"\n単位行列:\n{eye}")

### 1.3 テンソルの演算

In [ ]:
x = torch.tensor([[1., 2.], [3., 4.]])
y = torch.tensor([[5., 6.], [7., 8.]])

# 要素ごとの演算
print(f"x + y (要素ごとの加算):\n{x + y}")
print(f"\nx * y (要素ごとの乗算):\n{x * y}")

# 行列積（Transformerで最重要！）
print(f"\nx @ y (行列積):\n{x @ y}")
print(f"torch.matmul(x, y) も同じ:\n{torch.matmul(x, y)}")

# スカラー演算
print(f"\nx * 2:\n{x * 2}")
print(f"\nx / 2:\n{x / 2}")

### 1.4 形状変換（reshape, view, transpose）

In [ ]:
x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"元のテンソル: {x}")
print(f"形状: {x.shape}")

# reshape: 形状を変更
x_reshaped = x.reshape(3, 4)
print(f"\nreshape(3, 4):\n{x_reshaped}")

# view: reshapeとほぼ同じ（メモリ連続性が必要）
x_viewed = x.view(4, 3)
print(f"\nview(4, 3):\n{x_viewed}")

# -1 は自動計算
x_auto = x.reshape(2, -1)  # 2行、列数は自動
print(f"\nreshape(2, -1): 形状={x_auto.shape}")

# transpose: 次元を入れ替え
y = torch.randn(2, 3, 4)
print(f"\n元の形状: {y.shape}")
print(f"transpose(1, 2): {y.transpose(1, 2).shape}  # (2, 4, 3)")

# permute: 任意の次元順に並び替え
print(f"permute(2, 0, 1): {y.permute(2, 0, 1).shape}  # (4, 2, 3)")

### 1.5 次元の追加・削除（unsqueeze, squeeze）

In [ ]:
x = torch.randn(3, 4)
print(f"元の形状: {x.shape}")

# unsqueeze: 次元を追加
x1 = x.unsqueeze(0)  # 先頭に次元追加
print(f"unsqueeze(0): {x1.shape}  # バッチ次元を追加")

x2 = x.unsqueeze(1)  # 1番目に次元追加
print(f"unsqueeze(1): {x2.shape}")

x3 = x.unsqueeze(-1)  # 末尾に次元追加
print(f"unsqueeze(-1): {x3.shape}")

# squeeze: サイズ1の次元を削除
y = torch.randn(1, 3, 1, 4)
print(f"\n元の形状: {y.shape}")
print(f"squeeze(): {y.squeeze().shape}  # サイズ1の次元をすべて削除")
print(f"squeeze(0): {y.squeeze(0).shape}  # 0番目の次元のみ削除")

---

## 2. 自動微分（Autograd）

PyTorchは**自動微分**機能を持っています。これにより、勾配（微分）を自動で計算できます。

In [ ]:
# requires_grad=True で勾配を追跡
x = torch.tensor([2.0, 3.0], requires_grad=True)
print(f"x = {x}")

# 計算グラフを構築
y = x ** 2  # y = [4, 9]
z = y.sum()  # z = 13
print(f"y = x^2 = {y}")
print(f"z = sum(y) = {z}")

# 逆伝播（勾配計算）
z.backward()

# dz/dx = 2x
print(f"\n勾配 dz/dx = {x.grad}")
print(f"期待値: 2 * x = {2 * x.detach()}")

### 2.1 勾配の流れを止める

In [ ]:
x = torch.randn(3, requires_grad=True)

# 方法1: with torch.no_grad()
with torch.no_grad():
    y = x * 2
    print(f"no_grad内: requires_grad = {y.requires_grad}")

# 方法2: detach()
z = x.detach()
print(f"detach後: requires_grad = {z.requires_grad}")

# 推論時は勾配計算を止める（メモリ節約、高速化）

---

## 3. nn.Moduleでモデルを作る

PyTorchでニューラルネットワークを作るには `nn.Module` を継承します。

### 基本構造

In [ ]:
class SimpleNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()  # 親クラスの初期化（必須！）
        
        # 層の定義
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        """順伝播を定義（必須！）"""
        x = self.fc1(x)      # 線形変換
        x = self.relu(x)     # 活性化関数
        x = self.fc2(x)      # 線形変換
        return x

# モデルのインスタンス化
model = SimpleNetwork(input_size=10, hidden_size=20, output_size=5)
print(model)

# 順伝播
x = torch.randn(3, 10)  # バッチサイズ3、入力次元10
output = model(x)       # model.forward(x) と同じ
print(f"\n入力形状: {x.shape}")
print(f"出力形状: {output.shape}")

### 3.1 super().__init__() の意味

`super().__init__()` は親クラス（`nn.Module`）の初期化を呼び出します。これにより：
- パラメータの自動登録
- `.to(device)` でのGPU移動
- `.train()` / `.eval()` モード切り替え

などの機能が使えるようになります。**必ず書く必要があります。**

### 3.2 nn.Sequential を使った簡潔な書き方

In [ ]:
# nn.Sequential: 層を順番に適用
simple_model = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 5)
)

print(simple_model)

x = torch.randn(3, 10)
output = simple_model(x)
print(f"\n出力形状: {output.shape}")

---

## 4. よく使う層

Transformerで使われる主要な層を理解しましょう。

### 4.1 nn.Linear（線形層・全結合層）

数式: $y = xW^T + b$

入力の各要素に重みを掛けて足し合わせ、バイアスを加えます。

In [ ]:
# nn.Linear(入力次元, 出力次元)
linear = nn.Linear(4, 3)

print(f"重み(weight)の形状: {linear.weight.shape}")
print(f"バイアス(bias)の形状: {linear.bias.shape}")

x = torch.randn(2, 4)  # バッチ2、入力4次元
y = linear(x)
print(f"\n入力形状: {x.shape}")
print(f"出力形状: {y.shape}")

# バイアスなしの場合
linear_no_bias = nn.Linear(4, 3, bias=False)
print(f"\nbias=False: バイアスは{linear_no_bias.bias}")

### 4.2 nn.Embedding（埋め込み層）

整数のインデックス（トークンID）を密なベクトルに変換します。

**本質はルックアップテーブル**：ID→ベクトルの対応表です。

In [ ]:
# nn.Embedding(語彙サイズ, 埋め込み次元)
vocab_size = 100  # 100種類の単語
embed_dim = 16    # 各単語を16次元ベクトルで表現

embedding = nn.Embedding(vocab_size, embed_dim)

print(f"埋め込み行列の形状: {embedding.weight.shape}")
print(f"  = (語彙サイズ, 埋め込み次元)")

# トークンID列を入力
token_ids = torch.tensor([[1, 5, 3, 2],   # 文1
                          [7, 2, 8, 0]])  # 文2

embedded = embedding(token_ids)
print(f"\n入力（トークンID）形状: {token_ids.shape}")
print(f"出力（埋め込みベクトル）形状: {embedded.shape}")
print(f"  = (バッチサイズ, シーケンス長, 埋め込み次元)")

# padding_idx: 特定のIDをゼロベクトルに固定
embedding_with_pad = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
print(f"\npadding_idx=0: ID=0は常にゼロベクトル")

### 4.3 nn.LayerNorm（層正規化）

各サンプルの特徴量を正規化（平均0、分散1に近づける）します。

学習を安定させ、収束を早める効果があります。

In [ ]:
# nn.LayerNorm(正規化する次元のサイズ)
d_model = 8
layer_norm = nn.LayerNorm(d_model)

x = torch.randn(2, 3, d_model)  # (バッチ, シーケンス長, 特徴次元)
normalized = layer_norm(x)

print(f"入力形状: {x.shape}")
print(f"出力形状: {normalized.shape}")

# 正規化の効果を確認
print(f"\n正規化前: 平均={x[0, 0].mean():.4f}, 標準偏差={x[0, 0].std():.4f}")
print(f"正規化後: 平均={normalized[0, 0].mean():.4f}, 標準偏差={normalized[0, 0].std():.4f}")

### 4.4 nn.Dropout（ドロップアウト）

ランダムに一部のニューロンを無効化します。過学習を防ぐ正則化手法です。

**重要**: 訓練時のみ適用され、推論時は無効になります。

In [ ]:
dropout = nn.Dropout(p=0.5)  # 50%の確率で無効化

x = torch.ones(2, 10)

# 訓練モード
dropout.train()
out_train = dropout(x)
print(f"訓練モード: {out_train}")
print(f"  → 一部が0になり、残りは1/(1-p)=2倍にスケール")

# 評価モード
dropout.eval()
out_eval = dropout(x)
print(f"\n評価モード: {out_eval}")
print(f"  → 何も変化しない")

---

## 5. 活性化関数の選び方

活性化関数はニューラルネットワークに**非線形性**を導入します。

線形変換だけでは、どれだけ層を重ねても1つの線形変換と等価になってしまいます。

In [ ]:
x = torch.linspace(-5, 5, 100)

fig, axes = plt.subplots(2, 3, figsize=(12, 6))

# ReLU: 最も基本的、勾配消失しにくい
axes[0, 0].plot(x, F.relu(x))
axes[0, 0].set_title('ReLU: max(0, x)')
axes[0, 0].axhline(y=0, color='k', linewidth=0.5)
axes[0, 0].axvline(x=0, color='k', linewidth=0.5)
axes[0, 0].grid(True, alpha=0.3)

# GELU: Transformerで使用、滑らか
axes[0, 1].plot(x, F.gelu(x))
axes[0, 1].set_title('GELU: Transformerで人気')
axes[0, 1].axhline(y=0, color='k', linewidth=0.5)
axes[0, 1].axvline(x=0, color='k', linewidth=0.5)
axes[0, 1].grid(True, alpha=0.3)

# Sigmoid: 0-1に圧縮、二値分類の出力に
axes[0, 2].plot(x, torch.sigmoid(x))
axes[0, 2].set_title('Sigmoid: 0-1に圧縮')
axes[0, 2].axhline(y=0.5, color='r', linewidth=0.5, linestyle='--')
axes[0, 2].axvline(x=0, color='k', linewidth=0.5)
axes[0, 2].grid(True, alpha=0.3)

# Tanh: -1から1に圧縮
axes[1, 0].plot(x, torch.tanh(x))
axes[1, 0].set_title('Tanh: -1から1に圧縮')
axes[1, 0].axhline(y=0, color='k', linewidth=0.5)
axes[1, 0].axvline(x=0, color='k', linewidth=0.5)
axes[1, 0].grid(True, alpha=0.3)

# Softmax: 確率分布に変換
logits = torch.tensor([-1.0, 0.0, 1.0, 2.0])
probs = F.softmax(logits, dim=0)
axes[1, 1].bar(range(4), probs.numpy())
axes[1, 1].set_title('Softmax: 確率分布に変換')
axes[1, 1].set_xlabel('クラス')
axes[1, 1].set_ylabel('確率')

# SiLU/Swish: GELU類似、LLaMaなどで使用
axes[1, 2].plot(x, F.silu(x))
axes[1, 2].set_title('SiLU/Swish: x * sigmoid(x)')
axes[1, 2].axhline(y=0, color='k', linewidth=0.5)
axes[1, 2].axvline(x=0, color='k', linewidth=0.5)
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 活性化関数の選び方ガイド

| 活性化関数 | 用途 | 特徴 |
|-----------|------|------|
| **ReLU** | 隠れ層（デフォルト） | シンプル、高速、勾配消失しにくい |
| **GELU** | Transformer隠れ層 | 滑らか、BERT/GPTで使用 |
| **SiLU/Swish** | 最新のLLM | LLaMA, PaLMで使用 |
| **Sigmoid** | 二値分類の出力層 | 0-1に変換 |
| **Softmax** | 多クラス分類の出力層 | 確率分布に変換 |
| **Tanh** | 特殊な用途 | -1から1に変換 |

### 5.1 nn.関数 vs F.関数

PyTorchには2種類の使い方があります：

In [ ]:
x = torch.randn(3, 4)

# 方法1: nn.Module として（状態を持てる）
relu_module = nn.ReLU()
out1 = relu_module(x)

# 方法2: 関数として（F = torch.nn.functional）
out2 = F.relu(x)

print(f"nn.ReLU(): {out1.shape}")
print(f"F.relu(): {out2.shape}")
print(f"結果は同じ: {torch.allclose(out1, out2)}")

# どちらを使うべきか？
# - パラメータがある層（Linear, LayerNorm）→ nn.Module
# - パラメータがない関数（relu, softmax）→ どちらでもOK

---

## 6. 損失関数

損失関数は「モデルの予測がどれだけ間違っているか」を数値化します。

In [ ]:
# クロスエントロピー損失（分類タスク）
criterion = nn.CrossEntropyLoss()

# 予測（logits、softmax前の値）
predictions = torch.tensor([[2.0, 1.0, 0.1],   # クラス0が最大
                            [0.5, 2.5, 0.3]])  # クラス1が最大

# 正解ラベル
targets = torch.tensor([0, 1])  # サンプル1はクラス0、サンプル2はクラス1

loss = criterion(predictions, targets)
print(f"損失: {loss.item():.4f}")
print(f"  → 予測が正解に近いほど損失は小さい")

# 間違った予測の場合
wrong_targets = torch.tensor([2, 0])  # 間違ったラベル
wrong_loss = criterion(predictions, wrong_targets)
print(f"\n間違った予測の損失: {wrong_loss.item():.4f}")
print(f"  → 損失が大きくなる")

### 6.1 ignore_index（パディングを無視）

Transformerでは、パディングトークンの損失を計算しないようにします。

In [ ]:
# パディングを無視する損失関数
criterion = nn.CrossEntropyLoss(ignore_index=0)  # ID=0をパディングとして無視

# 予測（バッチ2、シーケンス長3、語彙サイズ5）
predictions = torch.randn(2, 3, 5)

# ターゲット（0はパディング）
targets = torch.tensor([[1, 2, 0],   # 3番目はパディング
                        [3, 0, 0]])  # 2,3番目はパディング

# 形状を変換して損失計算
loss = criterion(
    predictions.reshape(-1, 5),  # (バッチ*シーケンス, 語彙サイズ)
    targets.reshape(-1)          # (バッチ*シーケンス,)
)

print(f"損失（パディング無視）: {loss.item():.4f}")
print(f"  → ID=0の位置は損失計算に含まれない")

### 6.2 主な損失関数

| 損失関数 | 用途 | 入力 |
|---------|------|------|
| `nn.CrossEntropyLoss` | 多クラス分類 | logits (softmax前) |
| `nn.BCEWithLogitsLoss` | 二値分類 | logits (sigmoid前) |
| `nn.MSELoss` | 回帰 | 連続値 |
| `nn.L1Loss` | 回帰（外れ値に強い） | 連続値 |

---

## 7. 最適化アルゴリズム

最適化アルゴリズムは、損失を最小化するようにパラメータを更新します。

In [ ]:
# 簡単なモデル
model = nn.Linear(10, 5)

# Adam最適化（最も一般的）
optimizer = optim.Adam(model.parameters(), lr=0.001)

# SGD（基本的な勾配降下法）
# optimizer = optim.SGD(model.parameters(), lr=0.01)

# AdamW（重み減衰付きAdam、Transformerで推奨）
# optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)

print(f"最適化対象のパラメータ数: {sum(p.numel() for p in model.parameters())}")

### 7.1 学習率（Learning Rate）

学習率は「1回の更新でどれだけパラメータを変えるか」を決めます。

- **大きすぎる**: 発散する、収束しない
- **小さすぎる**: 学習が遅い、局所解に陥りやすい

一般的な値: `1e-3`（0.001）〜 `1e-4`（0.0001）

---

## 8. 学習ループの書き方

PyTorchでの学習ループの基本パターンを学びます。

In [ ]:
# 簡単な分類モデル
model = nn.Sequential(
    nn.Linear(4, 16),
    nn.ReLU(),
    nn.Linear(16, 3)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# ダミーデータ
X = torch.randn(100, 4)  # 100サンプル、4特徴
y = torch.randint(0, 3, (100,))  # 3クラス分類

# 学習ループ
num_epochs = 100
losses = []

for epoch in range(num_epochs):
    # 1. 訓練モードに設定
    model.train()
    
    # 2. 順伝播
    predictions = model(X)
    
    # 3. 損失計算
    loss = criterion(predictions, y)
    
    # 4. 勾配をリセット（重要！）
    optimizer.zero_grad()
    
    # 5. 逆伝播（勾配計算）
    loss.backward()
    
    # 6. パラメータ更新
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# 損失の推移をプロット
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('エポック')
plt.ylabel('損失')
plt.title('学習曲線')
plt.grid(True, alpha=0.3)
plt.show()

### 8.1 学習ループの各ステップ解説

```python
# 1. model.train() - 訓練モード
#    Dropout, BatchNormなどが訓練用の動作になる

# 2. predictions = model(X) - 順伝播
#    入力から出力を計算

# 3. loss = criterion(predictions, y) - 損失計算
#    予測と正解の差を計算

# 4. optimizer.zero_grad() - 勾配リセット
#    前回の勾配をクリア（しないと勾配が蓄積される）

# 5. loss.backward() - 逆伝播
#    損失から各パラメータの勾配を計算

# 6. optimizer.step() - パラメータ更新
#    勾配を使ってパラメータを更新
```

### 8.2 よくある間違い

In [ ]:
# ❌ 間違い1: zero_grad()を忘れる
# → 勾配が蓄積されて学習がおかしくなる

# ❌ 間違い2: backward()の後にzero_grad()を呼ぶ
# → 計算した勾配が消えてしまう

# ❌ 間違い3: step()の前にbackward()を忘れる
# → 勾配がゼロなので更新されない

# ✅ 正しい順序
# optimizer.zero_grad() → 順伝播 → loss.backward() → optimizer.step()

print("正しい順序:")
print("1. optimizer.zero_grad()  # 勾配クリア")
print("2. output = model(input)  # 順伝播")
print("3. loss = criterion(...)  # 損失計算")
print("4. loss.backward()        # 逆伝播")
print("5. optimizer.step()       # 更新")

---

## 9. 推論（評価）モード

学習後のモデルで予測を行う方法です。

In [ ]:
# 評価モードに設定
model.eval()

# 勾配計算を無効化（メモリ節約、高速化）
with torch.no_grad():
    # テストデータ
    X_test = torch.randn(10, 4)
    
    # 予測
    predictions = model(X_test)
    
    # 予測クラスを取得
    predicted_classes = predictions.argmax(dim=1)
    
    print(f"予測形状: {predictions.shape}")
    print(f"予測クラス: {predicted_classes}")

### 9.1 train() vs eval() の違い

| モード | Dropout | BatchNorm | 用途 |
|--------|---------|-----------|------|
| `model.train()` | 有効 | 統計更新 | 学習時 |
| `model.eval()` | 無効 | 固定統計 | 推論時 |

---

## 10. モデルの確認とデバッグ

モデルの構造やパラメータを確認する方法を学びます。

In [ ]:
# サンプルモデル
class SampleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(100, 32)
        self.linear1 = nn.Linear(32, 64)
        self.linear2 = nn.Linear(64, 10)
        self.layer_norm = nn.LayerNorm(64)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.linear1(x)
        x = self.layer_norm(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

model = SampleModel()

# モデル構造を表示
print("=" * 50)
print("モデル構造")
print("=" * 50)
print(model)

In [ ]:
# パラメータ数を確認
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"総パラメータ数: {count_parameters(model):,}")

# 各層のパラメータ数
print("\n各層のパラメータ:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape} = {param.numel():,}")

In [ ]:
# 入出力形状の確認
x = torch.randint(0, 100, (2, 5))  # バッチ2、シーケンス長5
print(f"入力形状: {x.shape}")

# 中間出力を確認したい場合
with torch.no_grad():
    embedded = model.embedding(x)
    print(f"Embedding後: {embedded.shape}")
    
    linear1_out = model.linear1(embedded)
    print(f"Linear1後: {linear1_out.shape}")
    
    output = model(x)
    print(f"最終出力: {output.shape}")

### 10.1 モデルの保存と読み込み

In [ ]:
# 保存（パラメータのみ）
# torch.save(model.state_dict(), 'model.pth')

# 読み込み
# model.load_state_dict(torch.load('model.pth'))

# state_dictの中身を確認
print("state_dict のキー:")
for key in model.state_dict().keys():
    print(f"  {key}")

---

## 11. GPUの使い方

PyTorchではGPU（CUDA/MPS）を使って計算を高速化できます。

In [ ]:
# デバイスの選択
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"CUDA GPU使用可能: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Apple Silicon GPU (MPS) 使用可能")
else:
    device = torch.device('cpu')
    print("CPU使用")

print(f"\n使用デバイス: {device}")

In [ ]:
# モデルをGPUに移動
model = SampleModel().to(device)
print(f"モデルのデバイス: 次のパラメータで確認")
print(f"  {next(model.parameters()).device}")

# データもGPUに移動
x = torch.randint(0, 100, (2, 5)).to(device)
print(f"\nデータのデバイス: {x.device}")

# 計算実行
output = model(x)
print(f"出力のデバイス: {output.device}")

### 11.1 デバイス間でのデータ移動

In [ ]:
# GPUからCPUに移動（結果の取得時など）
output_cpu = output.cpu()
print(f"CPUに移動: {output_cpu.device}")

# NumPy配列に変換（CPUのみ可能）
output_numpy = output_cpu.detach().numpy()
print(f"NumPy配列に変換: {type(output_numpy)}")

# 注意: GPUテンソルを直接NumPyに変換するとエラー
# output.numpy()  # ❌ エラー
# output.cpu().numpy()  # ✅ OK

---

## 12. Transformerコードとの対応

ここまで学んだ内容が、自作Transformerのコードでどのように使われているか確認しましょう。

### 12.1 Self-Attention（src/attention.py）

```python
class SelfAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()  # ← 親クラス初期化（必須）
        
        # Linear層でQ, K, Vを生成
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        
        self.d_model = d_model
    
    def forward(self, x, mask=None):
        # 線形変換
        Q = self.W_q(x)  # ← nn.Linearを適用
        K = self.W_k(x)
        V = self.W_v(x)
        
        # Attention計算
        scores = torch.matmul(Q, K.transpose(-2, -1))  # ← 行列積とtranspose
        scores = scores / (self.d_model ** 0.5)  # ← スケーリング
        
        weights = F.softmax(scores, dim=-1)  # ← softmaxで確率化
        output = torch.matmul(weights, V)  # ← 行列積
        
        return output, weights
```

### 12.2 Transformer（src/transformer.py）

```python
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, ...):
        super().__init__()
        
        # 埋め込み層（トークンID → ベクトル）
        self.src_embedding = nn.Embedding(src_vocab_size, d_model, padding_idx=0)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model, padding_idx=0)
        
        # 位置エンコーディング
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout)
        
        # Encoder/Decoder（nn.Moduleを継承したクラス）
        self.encoder = Encoder(...)
        self.decoder = Decoder(...)
        
        # 出力層（ベクトル → 語彙サイズの確率）
        self.output_projection = nn.Linear(d_model, tgt_vocab_size)
```

### 12.3 学習ループ（notebooks内のデモ）

```python
model = Transformer(...).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()  # ← 訓練モード
    
    for batch in dataloader:
        src, tgt_input, tgt_output = batch
        src, tgt_input, tgt_output = src.to(device), ...  # ← GPU移動
        
        optimizer.zero_grad()  # ← 勾配リセット
        
        output = model(src, tgt_input)  # ← 順伝播
        
        loss = criterion(
            output.reshape(-1, vocab_size),
            tgt_output.reshape(-1)
        )
        
        loss.backward()  # ← 逆伝播
        optimizer.step()  # ← 更新
```

---

## まとめ

### 最低限覚えるべきこと

1. **テンソル**: PyTorchの基本データ型、形状（shape）が重要
2. **nn.Module**: モデルの基底クラス、`__init__`と`forward`を定義
3. **nn.Linear**: 線形変換 $y = xW^T + b$
4. **nn.Embedding**: ID → ベクトルのルックアップテーブル
5. **活性化関数**: ReLU（デフォルト）、GELU（Transformer）
6. **損失関数**: CrossEntropyLoss（分類）
7. **最適化**: Adam, AdamW
8. **学習ループ**: zero_grad → forward → loss → backward → step
9. **評価モード**: model.eval() + torch.no_grad()
10. **GPU**: .to(device) でモデルとデータを移動

### 次のステップ

この基礎を理解したら、以下のノートブックでTransformerの実装を学んでいきましょう：

1. `01_self_attention_demo.ipynb` - Attention機構の理解
2. `02_multi_head_attention_demo.ipynb` - Multi-Head Attention
3. `tutorial_transformer.ipynb` - 完全なTransformerの使い方